# Moving from standard to stream learning

This tutorial is inspired by the [river library](https://riverml.xyz) examples and recipies. So, a closer look to those ones is encorage to any person who want to indepth on this particular topics.

Generaly speaking, nearly any approach in machine learning can be summarize in the following steps:

1. Defining the problem
    1. Identify the kind of problem (supervised vs un supervised, classification vs regression, etc.)
1. Loading the data
2. Preprocessing that data
    1. Feature extraction
    2. Feature selection
3. Fitting the model
    1. Ajusted the hyperparameters
4. Evaluate the model according certain measures

Let's see a classical example which ticks all those steps **using the scikit-learn library**:

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from rich import print


# Load the data
dataset = load_breast_cancer()
X, y = dataset.data, dataset.target

# Prepare the pipeline with the preprocessing steps and the model to be used
pipe = Pipeline([
    ('scale', StandardScaler()),                       # Prepapare the data centered in avg 0 and std 1
    ('extractor', PCA(0.95)),                          # Extract features that represents 0.95 of the variability
    ('classifier', LogisticRegression(solver='lbfgs')) # Last step the model to perform the classificaction
])

# Define a determistic cross-validation procedure
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Calculate the performance for each fold of the cross validation
scorer = make_scorer(roc_auc_score)
scores = cross_val_score(pipe, X, y, scoring=scorer, cv=cv)

# Display the average score and it's standard deviation
print(f'ROC AUC: {scores.mean():.4f} (± {scores.std():.4f})')

ROC AUC: 0.9776 (± 0.0165)

So, what is the problem with this approach, it has no problem but some downsize can be detected. First and foremost, if the data returned by `load_breast_cancer` had been too big for the memory of your computer, the program would have crashed. Although some techniques can be applied to minimize this threat, such as optimizing the typing, or the use of sparse storage for the data there is a limit to the optimizations available. If your dataset potentially can have millions of samples which can weigh hundreds of gigabytes, you will need some special hardware for sure. One solution is to do out-of-core learning; that is, algorithms that can learn by being presented the data in chunks or mini-batches, but potential issues arise from these considering the order and the local minima of the search space.

Another potential issue is the incorporation of new data into the model. Traditional approaches require starting from scratch with a new dataset result of the combination of the old data with the new samples available. This is particularly problematic in real-time applications where you have new data available now and then. In many real applications, the solution is to perform a continuous integration pipeline which can deploy a new model nearly several times per minute.

Finally, another problem of the traditional approach is the availability of the features. Some features might not be accessible at the particular point in time you are at, for example, a particular state of an object or an amount at a warehouse is not available in a week or month because the data is updated.

# Incremental learning

Incremental learning is also called online learning or stream learning, however, if you google "online learning", it is unlikely that the results are what you were looking for. Consequently, the terms "incremental learning" and "stream learning" are usually preferred. Behind incremental learning is a very clear idea which it is is to fit a model according to a continuous stream of data. In other words, the data isn't available in its entirety, but rather the observations are provided one by one. For example, imagine the previous classical example which, instead of the data set, we have a reference point providing one sample at a time. This can be simulated with a simple loop, such as:

In [2]:
for xi, yi in zip(X, y):
    pass

Since the data is already in memory, this is not the ideal scenario to exemplify it. However, keep in mind that in this particular case we are using each sample a single time. This can also be used in the same way by iterating on the results from a CSV file, a Kafka stream, an SQL query, etc.

One particular point to be aware of is the fact that in this example `xi` is an instance of `numpy.array` (data were loaded using scikit-learn library). By its design, the library that we are using for streaming learning, `river`, uses the class `dict` as the base of its behaviour. The authors of the library assume a point of view where each observation represents a single sample which could make sense from the point of view of a stream of data. It is no limitation but something to be aware of when performance is required. Remember that `dict` is implemented in **Python**, while the `numpy.array` is implemented at a low level in **C** and **Fortran**. One of the advantages of using `dict` is easy access to the features and a clearer program. It must be also noted that  **online processing is different to batch processing, in that vectorization doesn't bring any speedup**. However numeric processing libraries such as numpy actually bring too much overhead.

In [3]:
print(f"In numpy.array format:{xi}\n")
print(f"In dict format: {dict(zip(dataset.feature_names, xi))}")

In numpy.array format:[7.760e+00 2.454e+01 4.792e+01 1.810e+02 5.263e-02 4.362e-02 0.000e+00
 0.000e+00 1.587e-01 5.884e-02 3.857e-01 1.428e+00 2.548e+00 1.915e+01
 7.189e-03 4.660e-03 0.000e+00 0.000e+00 2.676e-02 2.783e-03 9.456e+00
 3.037e+01 5.916e+01 2.686e+02 8.996e-02 6.444e-02 0.000e+00 0.000e+00
 2.871e-01 7.039e-02]

In dict format: {'mean radius': 7.76, 'mean texture': 24.54, 'mean perimeter': 47.92, 'mean area': 181.0, 'mean 
smoothness': 0.05263, 'mean compactness': 0.04362, 'mean concavity': 0.0, 'mean concave points': 0.0, 'mean 
symmetry': 0.1587, 'mean fractal dimension': 0.05884, 'radius error': 0.3857, 'texture error': 1.428, 'perimeter 
error': 2.548, 'area error': 19.15, 'smoothness error': 0.007189, 'compactness error': 0.00466, 'concavity error': 
0.0, 'concave points error': 0.0, 'symmetry error': 0.02676, 'fractal dimension error': 0.002783, 'worst radius': 
9.456, 'worst texture': 30.37, 'worst perimeter': 59.16, 'worst area': 268.6, 'worst smoothness': 0.08996, 'worst 
compactness': 0.06444, 'worst concavity': 0.0, 'worst concave points': 0.0, 'worst symmetry': 0.2871, 'worst 
fractal dimension': 0.07039}

Although it is not complicated, `river` provides a convinient function to transform the datasets from `scikit-learn`to the required format. For the examples developed in this notebook, you can use the following

In [4]:
from river import stream
for xi, yi in stream.iter_sklearn_dataset(load_breast_cancer()):
    pass

Therefore, the change in our code seems pretty focalized and not quite important. By simply changing our well-known batches by the stream of data a lot of things cannot be done in the same way. For example, something so trivial and commonly done such as the standardisation of the data, i.e. scaling the data to have mean 0 and variance 1. This simple operation in the batched approach changes into something more complicated in the stream data because we don't know the values of the mean and the standard deviation before actually going through all the data. For this problem, a possible approach is to perform the first pass over the data to compute the necessary values and then scale the values during a second pass. However, this approach goes against out objective of only looking at the data once and many times it is not possible. So, how we can solve this, the answer comes in the shape o what is called computing running statistics or moving statistics. By using those we do not use the exact mean and standard deviation but an estimation which can be updated with each new value. For example, denoting the mean as $\mu_t$ and the count of element at any moment $n_t$, the mean can be easily updated at any moment by applying the following function:
$$
  n_{t+1} = n_t +1 \\    
  \mu_{t+1} = \mu_t +\frac{x - \mu_t}{n_{t+1}}    
$$

In the same way, for the variance($\sigma$), the previous formula can be completed with:
$$
  s_{t+1} = s_t + (x-\mu_t)\times(x-\mu_{t+1})\\
  \sigma_{t+1} = \frac{s_{t+1}}{n_{t+1}}
$$

where $s_t$  is a running sum of squares and $\sigma_t$ is the running variance at time $t$. These four formulae can be easily rewritten in Python for example, take the `mean radius` as our Guinea Pig:

In [5]:
#Let's inicialize the variables before any rolling calculation
n, mean, s, variance = 0, 0, 0, 0

for xi, yi in stream.iter_sklearn_dataset(load_breast_cancer()):
    n += 1
    mean_t = mean
    mean += (xi['mean radius'] - mean_t) / n
    s += (xi['mean radius'] - mean_t) * (xi['mean radius'] - mean)
    variance = s / n

    print(f'Running mean: {mean:.3f} - Running variance: {variance:.3f}')
    
print(f'Mean: {mean:.3f} - Variance: {variance:.3f}')

Running mean: 17.990 - Running variance: 0.000

Running mean: 19.280 - Running variance: 1.664

Running mean: 19.417 - Running variance: 1.147

Running mean: 17.418 - Running variance: 12.850

Running mean: 17.992 - Running variance: 11.600

...

Running mean: 14.123 - Running variance: 12.306

Running mean: 14.127 - Running variance: 12.295

Running mean: 14.139 - Running variance: 12.347

Running mean: 14.127 - Running variance: 12.397

Mean: 14.127 - Variance: 12.397

Now, compare the results with the ones implementations of `numpy`. 

In [6]:
import numpy as np
i = list(dataset.feature_names).index('mean radius')
print(f'True mean: {np.mean(X[:, i]):.3f}')
print(f'True variance: {np.var(X[:, i]):.3f}')

True mean: 14.127

True variance: 12.397

The results are identical with a key difference, the `numpy`implementation requires all data to be available for the calculation while the another is calculated as the problem progress in its development. Therefore, we should keep in mind that the results with a few instances are not going to be very accurate. 

So, once we can calculate different statistics on the data, the next step is to use those in order to normalize or standarize the data in a similar way as the batched approach. In `river`, several functions has been implemented in this sense in the `preprocessing` module. For example, to standarize all the features of the previous example we can perform:

In [7]:
from river.preprocessing import StandardScaler

scaler = StandardScaler()

for xi, yi in stream.iter_sklearn_dataset(load_breast_cancer()):
    scaler = scaler.learn_one(xi)

So, now that we are scaling the data, we can step forward and perform some machine learning approach. In this example, we are going to implement a linear regression based on an *stochastic gradient descent (SGD)*. The idea behind it is to optimize the output of the linear regression based on the maximum variance of the error made between the prediction and the true output. In this particular case, the *Squared Error* is the loss function in order to elude possible compensations due to signal change.

In [8]:
from river.linear_model import LogisticRegression
from river.optim import SGD

scaler = StandardScaler()
optimizer = SGD(lr=0.01)
log_reg = LogisticRegression(optimizer)

y_true = []
y_pred = []

for xi, yi in stream.iter_sklearn_dataset(load_breast_cancer(), shuffle=True, seed=42):

    # Scale the features
    xi_scaled = scaler.learn_one(xi).transform_one(xi)

    # Test the current model on the new "unobserved" sample
    yi_pred = log_reg.predict_proba_one(xi_scaled)
    # Train the model with the new sample
    log_reg.learn_one(xi_scaled, yi)

    # Store the truth and the prediction
    y_true.append(yi)
    y_pred.append(yi_pred[True])

print(f'ROC AUC: {roc_auc_score(y_true, y_pred):.4f}')

ROC AUC: 0.9896

The results seem to be a little better than the ones from `scikit-learn`. Let's make a proper comparison, to do that, we should use the same CV folds for each approach and compare the results. Although, we could define the same process quite easily, in order to make them completely comparable, `river` provides a module called `compat` which improves the compatibility with other Python libraries such as sklearn. So, we can use the function `convert_river_to_sklearn` to obtain an object perfectly compatible with the functions of `scikit-learn`.

In [9]:
from river.compose import Pipeline
from river.compat import convert_river_to_sklearn


# We define a Pipeline, becasuse we need a single object 
model = Pipeline(
    ('scale', StandardScaler()),
    ('ml_model', LogisticRegression())
)

# This funtion returns an object of type SKLRegressorWrapper 
# which is compatible with the signature of sklearn
model = convert_river_to_sklearn(model)

# Now, we can proceed using the cross_val_score from 
# sklearn with the River wrapped model
scores = cross_val_score(model, X, y, scoring=scorer, cv=cv)

# Let's compare the results
print(f'ROC AUC: {scores.mean():.4f} (± {scores.std():.4f})')

ROC AUC: 0.9639 (± 0.0158)

Although they are lower than the previous test, the results are comparable to the batch learning approach. 

# Pipelines

Although it has been barely introduce in the previous section, many times the "flow" of information in a machine learning approach can be modeled as a sequence of steps. In order to perform this operation, some libreries such as `scikit=learn` or `pandas` have the definition of objects to contain this type of definition. 

However, in practice, you are going to see very few use of the pipes because, for most developers, it is easier to think their workflows in a procedural way instead of a declarative one. The main reason is due to the mayority of batch machine learning, however, when we talk about online learning, pipelines seems the more natural way to reason. Let's compare the same process perform with a procedural approach and with a declarative one which is based on pipelines.

Now we are going to change the problem for one of Kaggle Competitions. 

In [10]:
from rich import print
from river.datasets import Restaurants

data = Restaurants()

print(data)

Data from the Kaggle Recruit Restaurants challenge.

The goal is to predict the number of visitors in each of 829 Japanese restaurants over a priod
of roughly 16 weeks. The data is ordered by date and then by restaurant ID.

      Name  Restaurants                                                               
      Task  Regression                                                                
   Samples  252,108                                                                   
  Features  7                                                                         
    Sparse  False                                                                     
      Path  /home/david/river_data/Restaurants/kaggle_recruit_restaurants.csv         
       URL  https://maxhalford.github.io/files/datasets/kaggle_recruit_restaurants.zip
      Size  27.54 MB                                                                  
Downloaded  True

Lets check the structure of the data:

In [11]:
print(next(iter(data)))

(
    {
        'store_id': 'air_04341b588bde96cd',
        'date': datetime.datetime(2016, 1, 1, 0, 0),
        'is_holiday': True,
        'genre_name': 'Izakaya',
        'area_name': 'Tōkyō-to Nerima-ku Toyotamakita',
        'latitude': 35.7356234,
        'longitude': 139.6516577
    },
    10
)

Lets begging with the procedural approach 

In [12]:
from river import feature_extraction, linear_model, metrics, preprocessing, stats, utils

# Let's create the average of the last 7, 14 and 21 days
#TargetAgg: Computes a streaming aggregate of the target values
features = (
    feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 7),target_name="last_7_mean"),
    feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 14),target_name="last_14_mean"),
    feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 21),target_name="last_21_mean")
)

scaler = preprocessing.StandardScaler()
model = linear_model.LinearRegression()
metric = metrics.MAE()

for x, y in data:

    # Derive date features
    x['weekday'] = x['date'].weekday()
    x['is_weekend'] = x['date'].weekday() in (5, 6)

    # Process the rolling means of the target  
    for mean_f in features:
        
        x = {**x, **mean_f.transform_one(x)}#** unzip a dictionary. In this way, it is concatenating two dicts   
        mean_f.learn_one(x, y)
   
    # Remove the key/value pairs that aren't features
    for key in ['store_id', 'date', 'genre_name', 'area_name', 'latitude', 'longitude']:
        x.pop(key)

    # Rescale the data
    x = scaler.learn_one(x).transform_one(x)

    # Fit the linear regression
    y_pred = model.predict_one(x)
    model.learn_one(x, y)

    # Update the metric using the out-of-fold prediction
    metric.update(y, y_pred)

print(metric)

MAE: 8.465114

In [13]:
#check the last sample
print(x)

{
    'is_holiday': -0.23103573677646685,
    'weekday': 1.0292832579142892,
    'is_weekend': 1.6249280076334165,
    'last_7_mean_mean_by_store_id': -1.350231449980909,
    'last_14_mean_mean_by_store_id': -1.4125913815779152,
    'last_21_mean_mean_by_store_id': -1.3980979075298516
}

Now, lets rewrite the same code but with an approach more declarative based on the steps od the pipeline

In [14]:
from river import compose

# funtion to transform the data in the same features used previously
def get_date_features(x):
    weekday =  x['date'].weekday()
    return {'weekday': weekday, 'is_weekend': weekday in (5, 6)}

# Build the pipeline with the same steps that has been previously done
model = compose.Pipeline(
    ('features', compose.TransformerUnion(
        ('date_features', compose.FuncTransformer(get_date_features)),
        ('last_7_mean', feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(),7),target_name="last_7_mean")),
        ('last_14_mean', feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(),14), target_name="last_14_mean")),
        ('last_21_mean', feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(),21), target_name="last_21_mean"))
    )),
    ('drop_non_features', compose.Discard('store_id', 'date', 'genre_name', 'area_name', 'latitude', 'longitude')),
    ('scale', preprocessing.StandardScaler()),
    ('lin_reg', linear_model.LinearRegression())
)

metric = metrics.MAE()

for x, y in data:

    # Make a prediction without using the target
    y_pred = model.predict_one(x)

    # Update the model using the target
    model.learn_one(x, y)

    # Update the metric using the out-of-fold prediction
    metric.update(y, y_pred)

print(x)
print(metric)


{
    'store_id': 'air_fff68b929994bfbd',
    'date': datetime.datetime(2017, 4, 22, 0, 0),
    'is_holiday': False,
    'genre_name': 'Bar/Cocktail',
    'area_name': 'Tōkyō-to Nakano-ku Nakano',
    'latitude': 35.7081457,
    'longitude': 139.66628799999998
}

MAE: 8.38533

As you can see we have arrange all the feature calculation and transform in an object `TransformerUnion`. The idea is to been able to calculate all features in parallel for each element that is passed to this step. Additionaly, the `for-loop` has become an image of the typical steps inside a online learning system and, therefore, the built-in evaluation function can be used to simplify, evenmore, the code.

In [15]:
from river import evaluate

model = compose.Pipeline(
    ('features', compose.TransformerUnion(
        ('date_features', compose.FuncTransformer(get_date_features)),
        ('last_7_mean', feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 7),target_name="last_7_mean" )),
        ('last_14_mean', feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 14),target_name="last_14_mean")),
        ('last_21_mean', feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 21),target_name="last_21_mean"))
    )),
    ('drop_non_features', compose.Discard('store_id', 'date', 'genre_name', 'area_name', 'latitude', 'longitude')),
    ('scale', preprocessing.StandardScaler()),
    ('lin_reg', linear_model.LinearRegression())
)

evaluate.progressive_val_score(dataset=data, model=model, metric=metrics.MAE())

MAE: 8.38533

Although the code is perfectly fine like this, because now we can use the built-in functions of the library. Some additional simplifications can be done. First, and opposite to the pipelines in `scikit-learn`, the name of the steps is not necessary.

In [16]:
model = compose.Pipeline(
    compose.TransformerUnion(
        compose.FuncTransformer(get_date_features),
        feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 7),target_name="last_7_mean"),
        feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 14),target_name="last_14_mean"),
        feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 21),target_name="last_21_mean")
    ),
    compose.Discard('store_id', 'date', 'genre_name', 'area_name', 'latitude', 'longitude'),
    preprocessing.StandardScaler(),
    linear_model.LinearRegression()
)

evaluate.progressive_val_score(dataset=data, model=model, metric=metrics.MAE())


MAE: 8.38533

In this case the library will infer one based on the order of the operations provided.

The next simplification comes from the fact that the pipeline can be declare with mathematical operations. Fist, use `+` to declare a `TransformerUnion` object.

In [17]:
model = compose.Pipeline(
    compose.FuncTransformer(get_date_features) + \
    feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 7),target_name="last_7_mean") + \
    feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 14),target_name="last_14_mean") + \
    feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 21),target_name="last_21_mean"),

    compose.Discard('store_id', 'date', 'genre_name', 'area_name', 'latitude', 'longitude'),
    preprocessing.StandardScaler(),
    linear_model.LinearRegression()
)

evaluate.progressive_val_score(dataset=data, model=model, metric=metrics.MAE())


MAE: 8.38533

Likewhise we can use the `|` operator to assemble steps into a `Pipeline`.

In [18]:
model = (
    compose.FuncTransformer(get_date_features) +
    feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 7),target_name="last_7_mean") +
    feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 14),target_name="last_14_mean") +
    feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), 21),target_name="last_21_mean")
)

to_discard = ['store_id', 'date', 'genre_name', 'area_name', 'latitude', 'longitude']

model = model | compose.Discard(*to_discard)
model |= preprocessing.StandardScaler()
model |= linear_model.LinearRegression()

evaluate.progressive_val_score(dataset=data, model=model, metric=metrics.MAE())


MAE: 8.38533

One final simplyfication comes from the fact that `river`automatically encapsulates functions in the `FuncTransform` so the final declarative model could be something like:

In [19]:
model = get_date_features

for n in [7, 14, 21]:
    model += feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), n),target_name="last_"+str(n)+"_mean")

model |= compose.Discard(*to_discard)
model |= preprocessing.StandardScaler()
model |= linear_model.LinearRegression()

evaluate.progressive_val_score(dataset=data, model=model, metric=metrics.MAE(), print_every=20_000)

[20,000] MAE: 8.751958
[40,000] MAE: 8.905389
[60,000] MAE: 8.928629
[80,000] MAE: 8.648981
[100,000] MAE: 8.491317
[120,000] MAE: 8.421815
[140,000] MAE: 8.347086
[160,000] MAE: 8.332035
[180,000] MAE: 8.441779
[200,000] MAE: 8.359055
[220,000] MAE: 8.316414
[240,000] MAE: 8.369178


MAE: 8.38533

We have include an additional argument to see how the evaluation changes over time. It must be said that here we are not talking about perfomance but declaration and the way we think in the model. Both models, procedural and this one are perfectly fine. As final point in the pipelines, it should be mentioned that we can graphicaly explore the pipeline when it is made based on pipes

In [20]:
model

Pipeline (
  TransformerUnion (
    FuncTransformer (
      func="get_date_features"
    ),
    TargetAgg (
      by=['store_id']
      how=Rolling (
        obj=Mean ()
        window_size=7
      )
      target_name="last_7_mean"
    ),
    TargetAgg (
      by=['store_id']
      how=Rolling (
        obj=Mean ()
        window_size=14
      )
      target_name="last_14_mean"
    ),
    TargetAgg (
      by=['store_id']
      how=Rolling (
        obj=Mean ()
        window_size=21
      )
      target_name="last_21_mean"
    )
  ),
  Discard (
    area_name
    date
    genre_name
    latitude
    longitude
    store_id
  ),
  StandardScaler (
    with_std=True
  ),
  LinearRegression (
    optimizer=SGD (
      lr=Constant (
        learning_rate=0.01
      )
    )
    loss=Squared ()
    l2=0.
    l1=0.
    intercept_init=0.
    intercept_lr=Constant (
      learning_rate=0.01
    )
    clip_gradient=1e+12
    initializer=Zeros ()
  )
)

One important additional point to highlight is the fact that, in order to debug the behavior of the different steps the function `debug_one`can be used. Imagine that, in this case we train the model with the first 120,000 examples and we want to know what happens with the following one. The next piece of code made the trick:

In [21]:
import itertools

model = get_date_features

for n in [7, 14, 21]:
    model += feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), n),target_name="last_"+str(n)+"_mean")


model |= compose.Discard(*to_discard)
model |= preprocessing.StandardScaler()
model |= linear_model.LinearRegression()

for x, y in itertools.islice(data, 120_000):
    y_pred = model.predict_one(x)
    model.learn_one(x, y)

x, y = next(iter(data))
print(model.debug_one(x))

0. Input
--------
area_name: Tōkyō-to Nerima-ku Toyotamakita (str)
date: 2016-01-01 00:00:00 (datetime)
genre_name: Izakaya (str)
is_holiday: True (bool)
latitude: 35.73562 (float)
longitude: 139.65166 (float)
store_id: air_04341b588bde96cd (str)

1. Transformer union
--------------------
    1.0 get_date_features
    ---------------------
    is_weekend: False (bool)
    weekday: 4 (int)

    1.1 TargetAgg
    -------------
    last_7_mean_mean_by_store_id: 36.28571 (float)

    1.2 TargetAgg1
    --------------
    last_14_mean_mean_by_store_id: 36.07143 (float)

    1.3 TargetAgg2
    --------------
    last_21_mean_mean_by_store_id: 36.23810 (float)

is_weekend: False (bool)
last_14_mean_mean_by_store_id: 36.07143 (float)
last_21_mean_mean_by_store_id: 36.23810 (float)
last_7_mean_mean_by_store_id: 36.28571 (float)
weekday: 4 (int)

2. ~['area_name', 'date', 'genre_name', 'latitude', 'longitude', 'store_id']
----------------------------------------------------------------------------
is_weekend: False (bool)
last_14_mean_mean_by_store_id: 36.07143 (float)
last_21_mean_mean_by_store_id: 36.23810 (float)
last_7_mean_mean_by_store_id: 36.28571 (float)
weekday: 4 (int)

3. StandardScaler
-----------------
is_weekend: -0.61171 (float)
last_14_mean_mean_by_store_id: 1.30097 (float)
last_21_mean_mean_by_store_id: 1.32644 (float)
last_7_mean_mean_by_store_id: 1.28498 (float)
weekday: 0.51168 (float)

4. LinearRegression
-------------------
Name                            Value      Weight      Contribution  
                    Intercept    1.00000    18.99682       18.99682  
last_21_mean_mean_by_store_id    1.32644     9.48096       12.57591  
last_14_mean_mean_by_store_id    1.30097     2.72577        3.54615  
                      weekday    0.51168     5.61055        2.87078  
 last_7_mean_mean_by_store_id    1.28498     1.23553        1.58763  
                   is_weekend   -0.61171    -1.96213        1.20026  

Prediction: 40.77756

# The incremental learning with decision trees

Until this point we have focussed on techniques that can be used in a seamessly way with the incremental learning, such as the lineal or logistic regression. However, there are other techniques that can be adapted to follow this approach, although they require a bit more effort. 

For example, a very important family amoung the machine learning techniques is all Decision Tree related ones. Over the years, epecially when used in a aggregation, this kind of techniques have proven their versatility, flexibility and simplicity. However, the the biggest part of those techniques, the approach requieres several iterations over the data making them unsuitable for incremental learning. Due to this problem, several alternative approaches have arisen in order to take this kind of methodology to the stream learning. 

One of the most popular adaptations is Hoeffding Trees (HT) due to their properties similar to batched approaches, such as (C4.5/J48, CART, M5, etc.):
* Only require a single pass over the data
* (Theoretical) guarantees the convergency with their batch counterpart with enough observations and a stationary data distribution
* It can operate on scarce resources of memory and computation time.
* Some of their variations can deal with non-stationary distributions
In fact, when the spotlight is over the Explanaible Artificial Intelligence (XAI) this kind of techniques take the head over other approaches for its easier interpretation.

Lets take a look to an example of the use of this kind of technique. For this example, let's use same Restaurant problem that you have already downloaded in the previous section to analyse the result of a single tree.  

In [ ]:
%%time

from river import evaluate, metrics, stats, utils, feature_extraction, preprocessing, compose
from river.tree import HoeffdingTreeRegressor
import itertools

data = Restaurants()
to_discard = ['store_id', 'date', 'genre_name', 'area_name', 'latitude', 'longitude']
def get_date_features(x):
    weekday =  x['date'].weekday()
    return {'weekday': weekday, 'is_weekend': weekday in (5, 6)}

model = get_date_features

for n in [7, 14, 21]:
    model += feature_extraction.TargetAgg(by='store_id', how=utils.Rolling(stats.Mean(), n),target_name="last_"+str(n)+"_mean")

model |= compose.Discard(*to_discard)
model |= preprocessing.StandardScaler()
model |= HoeffdingTreeRegressor(grace_period=250) #Number of instances a leaf should observe between split attempts

for x, y in data:
    model.learn_one(x, y)


From this point we can now analyse the resulting model in different ways

In [ ]:
#object analysing
model['HoeffdingTreeRegressor']

That gives an idea of the internal parameters of the object, although we can ask for a summary

In [ ]:
#summary of the tree
model['HoeffdingTreeRegressor'].summary

In [ ]:
#Pandas data frame 
(model['HoeffdingTreeRegressor']).to_dataframe()

Last one could be interesting in a classification approach due to the ability to analyse the internal decision of the model and check the thresholds and weights given, however, in regression it is not useful, lets see an example with the classification

In [ ]:

from river import evaluate, metrics, stats, utils, feature_extraction, preprocessing, compose
from river.tree import HoeffdingTreeClassifier
from river.datasets import Phishing #
import itertools

data = Phishing()

model = HoeffdingTreeClassifier(grace_period=50)

for x, y in data:
    model.learn_one(x, y)
    
    
#summary of the tree
print(model.summary)
    
model.to_dataframe()

As it can be seen much useful than the output of the regression problem. The same can be said for the plotting of the tree.

In [ ]:
#plot like a tree
#be aware to have graphviz installed on the system
model.draw()

`River`has many implementatrions of the tree but as the authors of the library remember on their [page](https://riverml.xyz/0.14.0/recipes/on-hoeffding-trees/#1-trees-trees-everywhere-gardening-101-with-river), most of them can be organized in the following table.

| Name | Acronym | Task | Non-stationary data? | Comments | Source |
| :- | :-: | :- | :-: | :- | :-: |
| Hoeffding Tree Classifier | HTC | Classification | No | Basic HT for classification tasks | [[1]](https://dl.acm.org/doi/pdf/10.1145/347090.347107)
| Hoeffding Adaptive Tree Classifier | HATC | Classification | Yes | Modifies HTC by adding an instance of ADWIN to each node to detect and react to drift detection | [[2]](https://link.springer.com/chapter/10.1007/978-3-642-03915-7_22)
| Extremely Fast Decision Tree Classifier | EFDT | Classification | No | Deploys split decisions as soon as possible and periodically revisit decisions and redo them if necessary. Not as fast in practice as the name implies, but it tends to converge faster than HTC to the model generated by a batch DT | [[3]](https://dl.acm.org/doi/abs/10.1145/3219819.3220005)
| Hoeffding Tree Regressor | HTR | Regression | No | Basic HT for regression tasks. It is an adaptation of the [FIRT/FIMT](https://link.springer.com/article/10.1007/s10618-010-0201-y) algorithm that bears some semblance to HTC | [[4]](https://link.springer.com/article/10.1007/s10618-010-0201-y)
| Hoeffding Adaptive Tree Regressor | HATR | Regression | Yes | Modifies HTR by adding an instance of ADWIN to each node to detect and react to drift detection |
| incremental Structured-Output Prediction Tree Regressor| iSOUPT | Multi-target regression | No | Multi-target version of HTR | [[5]](https://link.springer.com/article/10.1007/s10844-017-0462-7)
| Label Combination Hoeffding Tree Classifier | LCHTC | Multi-label classification | No | Creates a numerical code for each combination of the binary labels and uses HTC to learn from this encoded representation. At prediction time, decodes the modified representation to obtain the original label set |  -


As we can see each variant has a very specific target in mind, although they might overlap sometimes.

# Reading data from  a file


By working with online learning, we expect to obtain data from a stream of data in order to feed our model. However, we typically use saved datasets to develop our basis model before deploying it in a on-line production  environment. So far, the examples we have seen used datasets already contained in the libraries we are studying. These datasets are provided by the library developers, so they fit very well with the shown examples. However,  we will usually need to load our own datasets. River provides different connectors to get data from different  data sources including csv files. River connectors generate iterators over datasets in order to create data streams to feed our online models.

In this example we are going to get data from a csv file to analyze the different problems we can get in this process. We are using the very well-know Titanic dataset downloaded from [Kaggle](https://www.kaggle.com/c/titanic/data). It must be noted that this example is focused only on reading and processing data from a file. The model used and its performance is secondary.

Despite the online learning scenario differs from the batch learning one, we still need to know the input data structure in order to adapt it to our model. We encourage the readers to check the data dictionary from the [Kaggle web page](https://www.kaggle.com/c/titanic/data).


First of all, we load the dataset with Pandas to check data characteristics

In [ ]:
import pandas as pd
dataset_path="./datasets/titanic.csv" #you must change this var whether the dataset is in another path
data=pd.read_csv(dataset_path)
print(data.head())
print(data.dtypes)
print(data.describe())


Despite the fact River already has a connector to iterate from Pandas DataFrame, in this example we are going to use the [one](https://riverml.xyz/0.14.0/api/stream/iter-csv/)  linked to csv files.

<code>iter_csv</code> creates a Python iterator. Each time we use `next` we get a new observation consisting  of a set of both inputs and outputs

In [ ]:
from river import stream
from rich import print
titanic = stream.iter_csv(dataset_path)
sample, target = next(titanic)
print(sample)
print(target)

There is no way to know which sample attributes are targets, so every one is loaded as an input. We must specify the target attribute names

In [ ]:
titanic = stream.iter_csv(dataset_path, target="Survived")
sample, target = next(titanic)
print("###Inputs###")
for key, value in sample.items():
    print(f"{key}={value}")
print("###Targets###")
print(f"Survived: {target} - {'yes' if target==1 else 'no'}")

## Numerical and categorical (nominal) attributes
Using  a River connector, all the loaded attributes are interpreted as strings by default (categorical attributes). We must use the `converters` parameter to modify this behaviour and assign the specific type to each attribute. We are also able to use  (lambda) functions to *convert* the value of the attributes


* converters (dict) – defaults to None

    All values in the CSV are interpreted as strings by default. You can use this parameter to cast values to the desired type. This should be a dict mapping feature names to callables used to parse their associated values. Note that a callable may be a type, such as float and int.
    
It must be noted that `convert` functions are executed for each iterator call. If some problem occurs during the data conversion of an specific attribute, we will get an exception. Taking into account that data are not loaded at the beginning, but are loaded one by one by the iterator, exceptions are be able to be thrown in any loop moment. An example can be observed using a `float` converter to assign the type of the *Age* attribute. There are some observations without *Age* information, so a ValueError exception will be throw when an empty *Age* value appears.  In this specific example, we used a function (`float_converter`) for dealing with bad type conversions. Moreover, we can use the `drop_nones` parameter to remove the attributes with None values that we inserted during the data type conversion process.


We should also realize that some attributes do not contain valuable information to improve a model. We can drop them when the dataset is loaded or removed in future pipeline steps.


* drop (List[str]) – defaults to None

    Fields to ignore.


In [ ]:
removed_attributes=['PassengerId', 'Name', 'Ticket','Fare',]
sex=lambda  g: 0 if g=="male" else 1
def float_converter(a):
    try:
        a=float(a)
    except ValueError:
        a=None
    return a

cabin=lambda  g: 1 if g!="" else 0 #return 1 if the passenger had a cabin. Otherwise 0
    


titanic = stream.iter_csv(dataset_path, target="Survived", drop_nones=True,\
                          converters={'Survived':int,'Pclass':int, 'Age':float_converter, 'SibSp':int,'Parch':int, 'Sex':sex, 'Cabin':cabin},\
                          drop=removed_attributes)



for sample, target in titanic:
    print(sample, 'Survived' if target==1 else 'Deceased')


### One hot encoding

Different models can be used to deal with the same type of ML problem. However, not all the models can deal with categorical attributes. A typical solution is the one-hot attribute encoding. Rivers provides a preprocessing function to online encode categorical attributes. It must be highlighted that this method does not need to know all the categories in advance. They are added on the fly.

Let's check this one-hot encoding funcion with the *Embarked* attribute

In [ ]:
from river import stream, compose, preprocessing
titanic = stream.iter_csv(dataset_path, target="Survived", drop_nones=True,\
                          converters={'Survived':int,'Pclass':int, 'Age':float_converter, 'SibSp':int,'Parch':int, 'Sex':sex, 'Cabin':cabin},\
                          drop=removed_attributes)



pp = compose.Select('Embarked') | preprocessing.OneHotEncoder(sparse=False)
for sample, target in titanic:
    pp = pp.learn_one(sample)
    print(pp.transform_one(sample))
    

#### Adding preprocessing operations to the pipeline
We can create a River pipeline with different preprocessing operations for preparing the observations for developing the model. We are able to include feature engineering, one-hot encoding, feature removing, etc.

In [ ]:
from river import stream, compose, preprocessing
removed_attributes=['PassengerId', 'Name', 'Ticket','Fare',]
sex=lambda  g: 0 if g=="male" else 1
def float_converter(a):
    try:
        a=float(a)
    except ValueError:
        a=None
    return a

cabin=lambda  g: 1 if g!="" else 0 #return 1 if the passenger had a cabin. Otherwise 0

def generate_new_attributes(x):#feature enginnering
    x["FirstClass"]=1 if x["Pclass"]==1 else 0
    x["FamilyMembers"]=x["SibSp"]+x["Parch"]+1
    return x

titanic = stream.iter_csv(dataset_path, target="Survived", drop_nones=True,\
                          converters={'Survived':int,'Pclass':int, 'Age':float_converter, 'SibSp':int,'Parch':int, 'Sex':sex, 'Cabin':cabin},\
                          drop=removed_attributes)

to_discard=['SibSp','Parch','Embarked',"FirstClass"]#feature removing

pp=compose.FuncTransformer(generate_new_attributes)\
            +(compose.Select('Embarked') | preprocessing.OneHotEncoder(sparse=False))\
            |compose.Discard(*to_discard)
                            
    

for sample, target in titanic:
    pp = pp.learn_one(sample)
    print(pp.transform_one(sample), 'Survived' if target==1 else 'Deceased')
    

#### Wrapping up
Finally, we are able to include every step in a pipeline with a model as we shown in previous examples. Just remember that this example is focused on reading and processing data from a file. The model used and its performance is secondary.

In [ ]:
from river import stream, compose, preprocessing, evaluate, metrics
from river.tree import HoeffdingTreeClassifier
removed_attributes=['PassengerId', 'Name', 'Ticket','Fare',]
sex=lambda  g: 0 if g=="male" else 1
def float_converter(a):
    try:
        a=float(a)
    except ValueError:
        a=None
    return a

cabin=lambda  g: 1 if g!="" else 0 #return 1 if the passenger had a cabin. Otherwise 0

def generate_new_attributes(x):#feature enginnering
    x["FirstClass"]=1 if x["Pclass"]==1 else 0
    x["FamilyMembers"]=x["SibSp"]+x["Parch"]+1
    return x

titanic = stream.iter_csv(dataset_path, target="Survived", drop_nones=True,\
                          converters={'Survived':int,'Pclass':int, 'Age':float_converter, 'SibSp':int,'Parch':int, 'Sex':sex, 'Cabin':cabin},\
                          drop=removed_attributes)

to_discard=['SibSp','Parch','Embarked',"FirstClass"]#feature removing

model=compose.FuncTransformer(generate_new_attributes)\
            +(compose.Select('Embarked') | preprocessing.OneHotEncoder(sparse=False))\
            |compose.Discard(*to_discard)

model|= HoeffdingTreeClassifier(grace_period=50)


print(evaluate.progressive_val_score(dataset=titanic, model=model, metric=metrics.Accuracy()))
model['HoeffdingTreeClassifier'].draw()
                            